In [1]:
%gui wx

from PYME.LMVis import VisGUI

/Users/zachcm/opt/miniconda3/envs/pyme/lib/python3.7/_collections_abc.py:702: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  return len(self._mapping)
/Users/zachcm/opt/miniconda3/envs/pyme/lib/python3.7/_collections_abc.py:720: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  yield from self._mapping
/Users/zachcm/Code/python-microscopy/PYME/misc/extraCMaps.py:73: MatplotlibDeprecationWarning: 
The revcmap function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use Colormap.reversed() instead.
  cmapdat_r = cm.revcmap(ndat[cmapname])
INFO:OpenGL.acceleratesupport:No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


Trying to load 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
Loaded 3rd party recipe module ch_shrinkwrap.recipe_modules.surface_fitting
quaternion module not found, disabling custom clip plane orientations
quaternion module not found, disabling custom clip plane orientations


In [ ]:
# Delay so wx can boot up
import time
time.sleep(4)

In [ ]:
pymevis = VisGUI.ipython_pymevisualize()
pipeline = pymevis.pipeline

In [ ]:
import numpy as np
from ch_shrinkwrap import membrane_mesh

# Load torus mesh with no hole in it
mesh = membrane_mesh.MembraneMesh.from_stl('hemo.stl')
max_iters = 1
step_size = 0.1
# mesh.kc = 1.0
attraction_weight = 0  # eV/nm
curvature_weight = 10
method = 'euler'

mesh.a = attraction_weight
mesh.c = curvature_weight
mesh.max_iter = max_iters
mesh.step_size = step_size

pts = np.load('hemo_pts.npy')
sigma = np.load('hemo_sigma.npy')

In [ ]:
# Display torus points
from PYME.IO import tabular

pipeline.addDataSource('points', tabular.mappingFilter({'x': pts[:,0],
                                                        'y': pts[:,1],
                                                        'z': pts[:,2]}))
pipeline.selectDataSource('points')
pymevis.add_pointcloud_layer(ds_name='points')

In [ ]:
from PYME.LMVis.layers.mesh import TriangleRenderLayer

# Display torus mesh

mesh_name = pipeline.new_ds_name('hemo')
surf_count=0
pipeline.recipe.namespace[mesh_name] = mesh
layer = TriangleRenderLayer(pipeline, dsname=mesh_name, method='wireframe', cmap = ['C', 'M', 'Y', 'R', 'G', 'B'][surf_count % 6])
pymevis.add_layer(layer)

In [ ]:
# punch a hole
mesh.delaunay_remesh(pts)